# Pipeline for Discovering and Understanding Agent Error Patterns(Using embedding model amd LLM)

In [ ]:
pip install umap-learn hdbscan openai sentence-transformers hf_transfer cuml-cu12 --extra-index-url=https://pypi.nvidia.com

In [ ]:
import pandas as pd

pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", 500)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)

path = "/workspace/all_prompts_with_role.csv"
df = pd.read_csv(path)
display(df.head(50))

,file_path,prompt,role
0,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"Guide a team migrating from AWS to Google Cloud Platform (GCP) for hosting microservices. Analyze the differences in service offerings, pricing models, reliability, and integrations for serverless compute, storage, and networking. Based on the team's specific requirements and current expertise, recommend whether the switch is worth it and propose a phased transition approach with clear milestones.",user
1,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"I'll help you create a comprehensive migration guide from AWS to GCP for microservices. Let me start by organizing this analysis and gathering the latest information about both platforms.\n\n\n<function_calls>\n<create-file file_path=""todo.md"">\n# AWS to GCP Migration Guide for Microservices\n\n## 1. Research Phase\n[ ] Research current AWS microservices ecosystem and services\n[ ] Research GCP microservices ecosystem and services \n[ ] Compare serverless compute offerings (AWS Lambda vs Go...",assistant
2,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<tool_result> <create-file> ToolResult(success=True, output=""File 'todo.md' created successfully."") </create-file> </tool_result>",user
3,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"Now I'll begin researching the current AWS and GCP microservices ecosystems to gather the most up-to-date information for this migration analysis.\n\n<function_calls>\n<web-search query=""AWS microservices serverless compute services 2025 Lambda ECS EKS Fargate"" num_results=""10"">\n</web-search>\n</function_calls>",assistant
4,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<tool_result> <web-search> ToolResult(success=True, output='[{""title"": ""AWS Fargate: The Complete Guide 2025 - Carmatec"", ""url"": ""https://www.carmatec.com/blog/aws-fargate-the-complete-guide/""}, {""title"": ""AWS ECS vs. EKS vs. Fargate: The Essential Guide - nOps"", ""url"": ""https://www.nops.io/blog/ecs-vs-eks-vs-fargate/""}, {""title"": ""Serverless Computing – Amazon Web Services"", ""url"": ""https://aws.amazon.com/serverless/""}, {""title"": ""Serverless ICYMI Q3 2025 | AWS Compute Blog"", ""url"": ""https:...",user
5,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<function_calls>\n<web-search query=""Google Cloud Platform microservices serverless services 2025 Cloud Functions Cloud Run Cloud Run Jobs"" num_results=""10"">\n</web-search>\n</function_calls>",assistant
6,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<tool_result> <web-search> ToolResult(success=True, output='[{""title"": ""Google Cloud Functions in 2025: What Teams Should Know"", ""url"": ""https://cloudchipr.com/blog/google-cloud-functions""}, {""title"": ""What Is Google Cloud Run? A Complete Guide to Serverless ..."", ""url"": ""https://sealos.io/blog/what-is-cloud-run""}, {""title"": ""Serverless in Google Cloud: What\'s Worth Exploring in 2025"", ""url"": ""https://discuss.google.dev/t/serverless-in-google-cloud-what-s-worth-exploring-in-2025/188500""}, {...",user
7,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<function_calls>\n<web-search query=""AWS vs GCP microservices pricing comparison 2025 serverless compute storage networking costs"" num_results=""10"">\n</web-search>\n</function_calls>",assistant
8,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-1

## 1. Extract Error Events
### Identify all prompts that contain potential errors (using keyword heuristics or simple rules).
### For each error, retrieve the previous assistant tool-call, and convert them into a structured JSON:
### - action_nl (natural-language intent, if any)
### - action_command (actual executed command)
### - result_error_snippet (core part of the error)

In [ ]:
import re
import json
import pandas as pd
from pathlib import Path

CSV_PATH = Path("/workspace/all_prompts_with_role.csv")
OUT_CSV_PATH = CSV_PATH.with_name(CSV_PATH.stem + "_with_action_error—3Col.csv")

TAG_PATTERN = re.compile(r"<[^>]*>")

FUNC_CALLS_BLOCK_PATTERN = re.compile(
    r"<function_calls>.*?</function_calls>",
    re.DOTALL,
)

EXEC_CMD_PATTERN = re.compile(
    r"<execute-command>(.*?)</execute-command>",
    re.DOTALL,
)

def strip_all_tags(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return TAG_PATTERN.sub("", text).strip()

def remove_function_calls_block(text: str) -> str:
    if not isinstance(text, str):
        return ""
    return FUNC_CALLS_BLOCK_PATTERN.sub("", text).strip()

def extract_execute_command(text: str) -> str:
    if not isinstance(text, str):
        return ""
    m = EXEC_CMD_PATTERN.search(text)
    if not m:
        return ""
    return m.group(1).strip()

# load CSV
#df = pd.read_csv(CSV_PATH)

new_col = [""] * len(df)

for i in range(len(df)):
    row = df.iloc[i]
    role = row.get("role", "")
    prompt = row.get("prompt", "")

    if (
        role == "user"
        and isinstance(prompt, str)
        and "<execute-command>" in prompt
        and "</execute-command>" in prompt
        and i > 0  
    ):
        prev_prompt = df.iloc[i - 1].get("prompt", "")

        action_language = remove_function_calls_block(prev_prompt)
        action_command = extract_execute_command(prev_prompt)
        error_text = strip_all_tags(prompt)

        data = {
            "action langauge": action_language,
            "action command": action_command,
            "error": error_text,
        }

        new_col[i] = json.dumps(data, ensure_ascii=False)


df["action_error"] = new_col

df.to_csv(OUT_CSV_PATH, index=False)
print(f"Done. Saved to: {OUT_CSV_PATH}")


NameError: name 'df' is not defined

## 2. Cluster Error Events to Discover Hidden Error Types
### - Build embeddings for each {action + result} pair.
### - Use UMAP + HDBSCAN to form clusters that naturally group similar failures.
### - Each cluster typically corresponds to a real error type (e.g., port conflict, missing file, permission issue, JSON parse error).

In [ ]:
import pandas as pd
path = "/workspace/all_prompts_with_role_with_action_error—3Col.csv"
df = pd.read_csv(path)
display(df.head(50))

,file_path,prompt,role,action_error
0,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"Guide a team migrating from AWS to Google Cloud Platform (GCP) for hosting microservices. Analyze the differences in service offerings, pricing models, reliability, and integrations for serverless compute, storage, and networking. Based on the team's specific requirements and current expertise, recommend whether the switch is worth it and propose a phased transition approach with clear milestones.",user,NaN
1,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"I'll help you create a comprehensive migration guide from AWS to GCP for microservices. Let me start by organizing this analysis and gathering the latest information about both platforms.\n\n\n<function_calls>\n<create-file file_path=""todo.md"">\n# AWS to GCP Migration Guide for Microservices\n\n## 1. Research Phase\n[ ] Research current AWS microservices ecosystem and services\n[ ] Research GCP microservices ecosystem and services \n[ ] Compare serverless compute offerings (AWS Lambda vs Go...",assistant,NaN
2,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<tool_result> <create-file> ToolResult(success=True, output=""File 'todo.md' created successfully."") </create-file> </tool_result>",user,NaN
3,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"Now I'll begin researching the current AWS and GCP microservices ecosystems to gather the most up-to-date information for this migration analysis.\n\n<function_calls>\n<web-search query=""AWS microservices serverless compute services 2025 Lambda ECS EKS Fargate"" num_results=""10"">\n</web-search>\n</function_calls>",assistant,NaN
4,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<tool_result> <web-search> ToolResult(success=True, output='[{""title"": ""AWS Fargate: The Complete Guide 2025 - Carmatec"", ""url"": ""https://www.carmatec.com/blog/aws-fargate-the-complete-guide/""}, {""title"": ""AWS ECS vs. EKS vs. Fargate: The Essential Guide - nOps"", ""url"": ""https://www.nops.io/blog/ecs-vs-eks-vs-fargate/""}, {""title"": ""Serverless Computing – Amazon Web Services"", ""url"": ""https://aws.amazon.com/serverless/""}, {""title"": ""Serverless ICYMI Q3 2025 | AWS Compute Blog"", ""url"": ""https:...",user,NaN
5,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<function_calls>\n<web-search query=""Google Cloud Platform microservices serverless services 2025 Cloud Functions Cloud Run Cloud Run Jobs"" num_results=""10"">\n</web-search>\n</function_calls>",assistant,NaN
6,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<tool_result> <web-search> ToolResult(success=True, output='[{""title"": ""Google Cloud Functions in 2025: What Teams Should Know"", ""url"": ""https://cloudchipr.com/blog/google-cloud-functions""}, {""title"": ""What Is Google Cloud Run? A Complete Guide to Serverless ..."", ""url"": ""https://sealos.io/blog/what-is-cloud-run""}, {""title"": ""Serverless in Google Cloud: What\'s Worth Exploring in 2025"", ""url"": ""https://discuss.google.dev/t/serverless-in-google-cloud-what-s-worth-exploring-in-2025/188500""}, {...",user,NaN
7,/Users/yu.yan/Documents/coding_project/data/75e553dc-1731-4aaa-8fbb-12e91ae63773/NinjaAiGateway_1760985497_chat_completion.json,"<function_calls>\n<web-search query=""AWS vs GCP microservices pricing comparison 2025 serverless compute storage networking costs"" num_results=""10"">\n</web-search>\n</function_calls>",assistant,NaN
8,/Users/yu.yan/Documents/

In [ ]:
df_filtered = df[df["action_error"].notna()]
display(df_filtered.head(50))

,file_path,prompt,role,action_error
41,/Users/yu.yan/Documents/coding_project/data/386d99a1-94fe-4d1f-b2e6-93bcfb92cec2/NinjaAiGateway_1762933311_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" Wed Nov 12 07:41:26 UTC 2025\\n"",\n ""exit_code"": 0,\n ""cwd"": ""/workspace""\n}') </execute-command> </tool_result>",user,"{""action langauge"": ""Let me get more specific current news headlines by searching for today's date."", ""action command"": ""date"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" Wed Nov 12 07:41:26 UTC 2025\\\\n\"",\\n \""exit_code\"": 0,\\n \""cwd\"": \""/workspace\""\\n}')""}"
54,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" Collecting pandas>=2.0.0 (from -r requirements.txt (line 1))\\n Downloading pandas-2.3.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)\\n \\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\\u2501\...",user,"{""action langauge"": """", ""action command"": ""pip install -r requirements.txt"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" Collecting pandas>=2.0.0 (from -r requirements.txt (line 1))\\\\n Downloading pandas-2.3.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)\\\\n \\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u2501\\\\u25..."
58,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" ============================================================\\n Sales Data Aggregation and Weekly PDF Report Generator\\n ============================================================\\n 2025-10-29 20:37:00,776 - INFO - SalesReportGenerator initialized with output directory: reports\\n \\n 1. Loading/Generating sales data...\\n 2025-10-29 20:37:00,776 - INFO - Generating sample sales data...\\n 2...",user,"{""action langauge"": ""## Data Processing &amp; Testing\n\nNow let me test the script to ensure it works correctly and generates the sample data and reports."", ""action command"": ""python sales_report_generator.py"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" ============================================================\\\\n Sales Data Aggregation and Weekly PDF Report Generator\\\\n ============================================================\\\\n 2025-10-29 20:37:00,..."
62,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" ============================================================\\n Sales Data Aggregation and Weekly PDF Report Generator\\n ============================================================\\n 2025-10-29 20:37:25,144 - INFO - SalesReportGenerator initialized with output directory: reports\\n \\n 1. Loading/Generating sales data...\\n 2025-10-29 20:37:25,144 - INFO - Generating sample sales data...\\n 2...",user,"{""action langauge"": ""Now let me run the script again to test the fix:"", ""action command"": ""python sales_report_generator.py"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" ============================================================\\\\n Sales Data Aggregation and Weekly PDF Report Generator\\\\n ============================================================\\\\n 2025-10-29 20:37:25,144 - INFO - S

#### 2.1 Only keep the prompt that may contain the error message

In [ ]:
import json
import pandas as pd

# Maybe threr wil be some more key words
ERROR_KEYWORDS = [
    "error", "exception", "failed", "fail", "traceback",
    "no such file", "not found", "file not found",
    "permission denied", "access denied",
    "address already in use",
    "fatal", "crash", "segmentation fault", "segfault",
    "cuda error", "out of memory", "oom",
    "timeout", "timed out",
    "connection refused", "connection reset",
    "cannot import", "module not found",
    "syntaxerror", "runtimeerror", "typeerror", "valueerror", 
    "undefined", "invalid",
    "broken pipe",
]

def looks_like_error(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.lower()
    return any(kw in t for kw in ERROR_KEYWORDS)

# df = pd.read_csv("xxx.csv")

def extract_error_field(x):

    if not isinstance(x, str):
        return ""
    try:
        data = json.loads(x)
        return data.get("error", "")
    except Exception:
        return ""

df["parsed_error"] = df["action_error"].apply(extract_error_field)
df["looks_like_error"] = df["parsed_error"].apply(looks_like_error)

df_errors = df[df["looks_like_error"] == True]

print(df_errors.shape)
df_errors.head()


(4052, 6)


,file_path,prompt,role,action_error,parsed_error,looks_like_error
58,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" ============================================================\\n Sales Data Aggregation and Weekly PDF Report Generator\\n ============================================================\\n 2025-10-29 20:37:00,776 - INFO - SalesReportGenerator initialized with output directory: reports\\n \\n 1. Loading/Generating sales data...\\n 2025-10-29 20:37:00,776 - INFO - Generating sample sales data...\\n 2...",user,"{""action langauge"": ""## Data Processing &amp; Testing\n\nNow let me test the script to ensure it works correctly and generates the sample data and reports."", ""action command"": ""python sales_report_generator.py"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" ============================================================\\\\n Sales Data Aggregation and Weekly PDF Report Generator\\\\n ============================================================\\\\n 2025-10-29 20:37:00,...","ToolResult(success=True, output='{\n ""output"": "" ============================================================\\n Sales Data Aggregation and Weekly PDF Report Generator\\n ============================================================\\n 2025-10-29 20:37:00,776 - INFO - SalesReportGenerator initialized with output directory: reports\\n \\n 1. Loading/Generating sales data...\\n 2025-10-29 20:37:00,776 - INFO - Generating sample sales data...\\n 2025-10-29 20:37:01,114 - INFO - ...",True
70,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" Traceback (most recent call last):\\n File \\""/workspace/demo_usage.py\\"", line 15, in <module>\\n import schedule\\n ModuleNotFoundError: No module named \'schedule\'\\n"",\n ""exit_code"": 1,\n ""cwd"": ""/workspace""\n}') </execute-command> </tool_result>",user,"{""action langauge"": ""Now let me run the demo to show all the functionality:"", ""action command"": ""python demo_usage.py"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" Traceback (most recent call last):\\\\n File \\\\\""/workspace/demo_usage.py\\\\\"", line 15, in \\\\n import schedule\\\\n ModuleNotFoundError: No module named \\'schedule\\'\\\\n\"",\\n \""exit_code\"": 1,\\n \""cwd\"": \""/workspace\""\\n}')""}","ToolResult(success=True, output='{\n ""output"": "" Traceback (most recent call last):\\n File \\""/workspace/demo_usage.py\\"", line 15, in \\n import schedule\\n ModuleNotFoundError: No module named \'schedule\'\\n"",\n ""exit_code"": 1,\n ""cwd"": ""/workspace""\n}')",True
84,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" # Sales Data Aggregation and Weekly PDF Reports\\n \\n ## Planning & Setup\\n - [ ] Design the script structure and requirements\\n - [ ] Set up necessary dependencies and libraries\\n - [ ] Create sample sales data structure\\n - [ ] Implement data aggregation functions\\n \\n ## Data Processing\\n - [ ] Create sales data reading/processing functions\\n - [ ] Implement total sales calcula...",user,"{""action langauge"": """", ""action command"": ""cat todo.md"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" # Sales Data Aggregation and Weekly PDF Reports\\\\n \\\\n ## Planning & Setup\\\\n - [ ] Design the script structure and requirements\\\\n - [ ] Set up necessary dependencies and libraries\\\\n - [ ] Create sample sales data structure\\\\n - [ ] Implement data aggregation functions\\\\n \\\\n ## Data Processing\\\\n - [ ] Create sales data readin

#### 2.2 Build embeddings for each {action + result} pair.

In [ ]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer("BAAI/bge-m3", device="cuda")

# ---- extrace text containing error ----

def build_error_text(row):
    ae = row["action_error"]
    if not isinstance(ae, str):
        return ""
    try:
        d = json.loads(ae)
    except:
        return ""

    lang = d.get("action langauge", "")
    cmd = d.get("action command", "")
    err = d.get("error", "")

    return f"[LANG] {lang}\n[CMD] {cmd}\n[ERR] {err}"


df_errors = df[df["looks_like_error"] == True].copy()
df_errors["error_text"] = df_errors.apply(build_error_text, axis=1)


In [ ]:
# ---- get embedding ----
def get_embedding(text):
    if not isinstance(text, str):
        text = ""
    return model.encode(text, normalize_embeddings=True)

all_embeddings = []
for t in tqdm(df_errors["error_text"], desc="Embedding"):
    all_embeddings.append(get_embedding(t))

X = np.array(all_embeddings)
print("embedding shape:", X.shape)

Embedding: 100%|██████████| 4052/4052 [01:32<00:00, 43.84it/s]

embedding shape: (4052, 1024)


#### 2.3 Use UMAP + HDBSCAN to form clusters that naturally group similar failures

In [ ]:
# ---- UMAP ----
try:
    from cuml.manifold import UMAP
    umap_model = UMAP(n_neighbors=15, min_dist=0.05, metric="cosine")
    X_umap = umap_model.fit_transform(X)
except:
    import umap
    umap_model = umap.UMAP(n_neighbors=15, min_dist=0.05, metric="cosine")
    X_umap = umap_model.fit_transform(X)

In [ ]:
# ---- HDBSCAN ----
try:
    from cuml.cluster import HDBSCAN
    clusterer = HDBSCAN(min_cluster_size=20, metric="euclidean")
    labels = clusterer.fit_predict(X_umap)
except:
    import hdbscan
    clusterer = hdbscan.HDBSCAN(min_cluster_size=20, metric="euclidean")
    labels = clusterer.fit_predict(X_umap)

df_errors["cluster"] = labels

/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
df_errors.to_csv("/workspace/df_errors.csv", index=False, encoding="utf-8")

## 3. Summarize and Name Each Error Cluster using LLM

In [ ]:
import pandas as pd
path = "/workspace/df_errors.csv"
df_errors = pd.read_csv(path)
display(df_errors.head(50))

,file_path,prompt,role,action_error,parsed_error,looks_like_error,error_text,cluster
0,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" ============================================================\\n Sales Data Aggregation and Weekly PDF Report Generator\\n ============================================================\\n 2025-10-29 20:37:00,776 - INFO - SalesReportGenerator initialized with output directory: reports\\n \\n 1. Loading/Generating sales data...\\n 2025-10-29 20:37:00,776 - INFO - Generating sample sales data...\\n 2...",user,"{""action langauge"": ""## Data Processing &amp; Testing\n\nNow let me test the script to ensure it works correctly and generates the sample data and reports."", ""action command"": ""python sales_report_generator.py"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" ============================================================\\\\n Sales Data Aggregation and Weekly PDF Report Generator\\\\n ============================================================\\\\n 2025-10-29 20:37:00,...","ToolResult(success=True, output='{\n ""output"": "" ============================================================\\n Sales Data Aggregation and Weekly PDF Report Generator\\n ============================================================\\n 2025-10-29 20:37:00,776 - INFO - SalesReportGenerator initialized with output directory: reports\\n \\n 1. Loading/Generating sales data...\\n 2025-10-29 20:37:00,776 - INFO - Generating sample sales data...\\n 2025-10-29 20:37:01,114 - INFO - ...",True,"[LANG] ## Data Processing &amp; Testing\n\nNow let me test the script to ensure it works correctly and generates the sample data and reports.\n[CMD] python sales_report_generator.py\n[ERR] ToolResult(success=True, output='{\n ""output"": "" ============================================================\\n Sales Data Aggregation and Weekly PDF Report Generator\\n ============================================================\\n 2025-10-29 20:37:00,776 - INFO - SalesReportGenerator initializ...",50
1,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "" Traceback (most recent call last):\\n File \\""/workspace/demo_usage.py\\"", line 15, in <module>\\n import schedule\\n ModuleNotFoundError: No module named \'schedule\'\\n"",\n ""exit_code"": 1,\n ""cwd"": ""/workspace""\n}') </execute-command> </tool_result>",user,"{""action langauge"": ""Now let me run the demo to show all the functionality:"", ""action command"": ""python demo_usage.py"", ""error"": ""ToolResult(success=True, output='{\\n \""output\"": \"" Traceback (most recent call last):\\\\n File \\\\\""/workspace/demo_usage.py\\\\\"", line 15, in \\\\n import schedule\\\\n ModuleNotFoundError: No module named \\'schedule\\'\\\\n\"",\\n \""exit_code\"": 1,\\n \""cwd\"": \""/workspace\""\\n}')""}","ToolResult(success=True, output='{\n ""output"": "" Traceback (most recent call last):\\n File \\""/workspace/demo_usage.py\\"", line 15, in \\n import schedule\\n ModuleNotFoundError: No module named \'schedule\'\\n"",\n ""exit_code"": 1,\n ""cwd"": ""/workspace""\n}')",True,"[LANG] Now let me run the demo to show all the functionality:\n[CMD] python demo_usage.py\n[ERR] ToolResult(success=True, output='{\n ""output"": "" Traceback (most recent call last):\\n File \\""/workspace/demo_usage.py\\"", line 15, in \\n import schedule\\n ModuleNotFoundError: No module named \'schedule\'\\n"",\n ""exit_code"": 1,\n ""cwd"": ""/workspace""\n}')",60
2,/Users/yu.yan/Documents/coding_project/data/07aed6b5-b250-4e92-9f6e-7168c93bb9a0/NinjaAiGateway_1761770680_chat_completion.json,"<tool_result> <execute-command> ToolResult(success=True, output='{\n ""output"": "

In [ ]:
import json
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch      
import gc         

model_id = "Qwen/Qwen3-8B"
cache_dir = "/workspace/model"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    #load_in_4bit=True,
    trust_remote_code=True,
    cache_dir = "/workspace/model"
)

gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.2,
    top_p=0.9,
    do_sample=False,
    return_full_text=False,
)

SYSTEM_PROMPT = """You are an expert log and error analyst.

You will be given multiple log snippets. 
All of them come from the SAME error cluster (same underlying error type).

Each log snippet has:
- [LANG]: the natural language description of what the agent wanted to do
- [CMD]: the actual command or tool call that was executed
- [ERR]: the error message or traceback

Your tasks:

1. First, infer the most likely ROOT CAUSE shared by these errors.
2. THEN, based on this root cause, propose a SHORT error type name in UPPER_CASE_WITH_UNDERSCORES (e.g. PORT_IN_USE, FILE_NOT_FOUND).
3. Provide 2–4 short EXAMPLE error messages or patterns that are representative of this error type.

IMPORTANT OUTPUT RULES (READ CAREFULLY):

- You MUST reply in valid JSON only.
- Do NOT add any explanation, commentary, or thinking process.
- Do NOT wrap the JSON in Markdown.
- Do NOT output anything before or after the JSON.
- The JSON MUST follow EXACTLY this schema:

{
  "cluster_name": "give a error type name",
  "root_cause": "One or two sentences describing the underlying cause in natural language.",
  "summary": [
    "example error message 1 ...",
    "example error message 2 ...",
    "example error message 3 ..."
  ]
}

Where:
- "cluster_name" is a short error type name in UPPER_CASE_WITH_UNDERSCORES, derived from the root_cause.
- "root_cause" is a concise description of what is going wrong technically.
- "summary" is an array of 2–4 representative error message examples.

Only output the JSON object. Do NOT output anything else.

Logs:
"""


MAX_CHARS_PER_LOG = 10000 

def truncate_text(s, max_chars=MAX_CHARS_PER_LOG):
    if not isinstance(s, str):
        return ""
    s = s.strip()
    if len(s) <= max_chars:
        return s
    return s[:max_chars]


def summarize_one_cluster(cluster_id, df_cluster, num_samples=25):
    raw_texts = df_cluster["error_text"].sample(
        n=min(num_samples, len(df_cluster)),
        random_state=42
    ).tolist()

    sample_texts = [truncate_text(t) for t in raw_texts]

    joined_logs = "\n\n---\n\n".join(sample_texts)

    prompt = SYSTEM_PROMPT + joined_logs

    out = gen_pipe(
        prompt,
        do_sample=False
    )[0]["generated_text"]

    #completion = out[len(prompt):].strip()
    completion = out.strip()

 
    try:
        start = completion.find("{")
        end = completion.rfind("}")
        if start != -1 and end != -1:
            json_str = completion[start:end+1]
            info = json.loads(json_str)
        else:
            info = {
                "cluster_name": f"CLUSTER_{cluster_id}",
                "root_cause": completion[:],
                "summary": completion[:],
            }
    except Exception:
        info = {
            "cluster_name": f"CLUSTER_{cluster_id}",
            "root_cause": completion[:],
            "summary": completion[:],
        }

    return info

cluster_sizes = df_errors.groupby("cluster").size()
cluster_infos = []

for idx, (cid, size) in enumerate(cluster_sizes.items(), start=1):
    
    if cid == -1:
        continue
    if size <= 60:
        continue

    df_c = df_errors[df_errors["cluster"] == cid]
    print(f"\n=== Summarizing cluster {cid}, size={size} ===")

    info = summarize_one_cluster(cid, df_c, num_samples=10)
    info["cluster_id"] = cid
    info["size"] = size
    cluster_infos.append(info)

    if idx % 5 == 0:
        print(f"[GC] Finished {idx} clusters, running gc + torch.cuda.empty_cache()")
        gc.collect()
        torch.cuda.empty_cache()

df_cluster_info = pd.DataFrame(cluster_infos)
print(df_cluster_info)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0



=== Summarizing cluster 1, size=406 ===

=== Summarizing cluster 2, size=137 ===

=== Summarizing cluster 8, size=64 ===
[GC] Finished 10 clusters, running gc + torch.cuda.empty_cache()

=== Summarizing cluster 18, size=65 ===
[GC] Finished 20 clusters, running gc + torch.cuda.empty_cache()

=== Summarizing cluster 23, size=168 ===
[GC] Finished 25 clusters, running gc + torch.cuda.empty_cache()

=== Summarizing cluster 24, size=99 ===

=== Summarizing cluster 27, size=91 ===

=== Summarizing cluster 32, size=67 ===

=== Summarizing cluster 49, size=146 ===

=== Summarizing cluster 50, size=66 ===

=== Summarizing cluster 58, size=71 ===
[GC] Finished 60 clusters, running gc + torch.cuda.empty_cache()

=== Summarizing cluster 60, size=107 ===

=== Summarizing cluster 61, size=63 ===
                   cluster_name  \
0                     CLUSTER_1   
1                     CLUSTER_2   
2            ENDPOINT_NOT_FOUND   
3             TEST_CASE_FAILURE   
4                    CLUSTER_2

In [ ]:
import pandas as pd
pd.set_option("display.width", 200)
pd.set_option("display.max_colwidth", 500)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 200)
df_cluster_info

,cluster_name,root_cause,summary,cluster_id,size
0,CLUSTER_1,"---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute session command: ')\n\n---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute session command: ')\n\n---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute session command: ')\n\n---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute...","---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute session command: ')\n\n---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute session command: ')\n\n---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute session command: ')\n\n---\n\n[LANG] \n[CMD] \n[ERR] ToolResult(success=False, output='Error executing command: Failed to execute...",1,406
1,CLUSTER_2,"Based on the provided logs, all entries show the same error message: `OSError: [Errno 98] Address already in use`. This indicates that the application is attempting to bind to a network port that is already occupied by another process. The error occurs during the initialization of an HTTP server, specifically when trying to bind the socket to a server address.\n\n### Root Cause\nThe root cause is that the application is trying to use a port that is already being used by another process, lead...","Based on the provided logs, all entries show the same error message: `OSError: [Errno 98] Address already in use`. This indicates that the application is attempting to bind to a network port that is already occupied by another process. The error occurs during the initialization of an HTTP server, specifically when trying to bind the socket to a server address.\n\n### Root Cause\nThe root cause is that the application is trying to use a port that is already being used by another process, lead...",2,137
2,ENDPOINT_NOT_FOUND,The application is attempting to access API endpoints that do not exist or are not properly configured in the server.,"[Task not found, Cannot GET /api/health, 403 Forbidden, The requested endpoint does not exist]",8,64
3,TEST_CASE_FAILURE,"The tests are failing due to incorrect implementation of the function, either due to logical errors in the algorithm or improper error handling that doesn't match expected behavior.","[Test case 1: [1, 2, 3, 4, 5] - Average: 3.0, Test case 2: [1.5, 2.5, 3.5, 4.5, 5.5] - Average: 3.5, Error test case 1: [] - Cannot calculate average of an empty list]",18,65
4,CLUSTER_23,"Based on the provided logs, I'll analyze the error patterns and determine the root cause.\n\nLooking at the logs, I notice several issues:\n1. Multiple errors related to ""tuple index out of range"" when processing EPUB files\n2. ""NoneType' object has no attribute 'get'"" errors during content extraction\n3. ""ImportError: cannot import name 'Epub'"" from a library\n4. ""SyntaxError: unterminated string literal"" in the code\n5. ""No content found in body"" when trying to extract main content\n\nThe ...","Based on the provided logs, I'll analyze the error patterns and determine the root cause.\n\nLooking at the logs, I notice several issues:\n1. Multiple errors related to ""tuple index out of range"" when processing EPUB files\n2. ""NoneType' object has no attribute 'get'"" errors during content extraction\n3. ""ImportError: cannot import name 'Epub'"" from a library\n4. ""SyntaxError: unterminated string literal"" in the code\n5. ""No content found in body"" when trying to extract main content\n\nThe ...",23,168
5,CLUSTER_24,"After analyzing the provided logs, I identified a common root cause across the errors. The errors primarily involve issues with the wkhtmltopdf tool, specifically related to its integration with Qt and handling of certain comman

In [ ]:
# clear cache
del gen_pipe
del model
del tokenizer
import gc, torch
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()